In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w6p1/sample_submission.csv
/kaggle/input/2022-ai-w6p1/pca_test.csv
/kaggle/input/2022-ai-w6p1/pca_train.csv
/kaggle/input/2022-ai-w6p1/train_label.csv


In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
x_train = pd.read_csv("/kaggle/input/2022-ai-w6p1/pca_train.csv")
y_train = pd.read_csv("/kaggle/input/2022-ai-w6p1/train_label.csv")
x_test = pd.read_csv("/kaggle/input/2022-ai-w6p1/pca_test.csv")
submit = pd.read_csv("/kaggle/input/2022-ai-w6p1/sample_submission.csv")

In [4]:
#from sklearn.model_selection import train_test_split

#train_input, test_input, train_target, test_target = train_test_split(x_train, y_train)

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
y_train = y_train['label']

In [7]:
x_train = torch.FloatTensor(np.array(x_train)).cuda()
y_train = torch.FloatTensor(np.array(y_train))
x_test = torch.FloatTensor(np.array(x_test)).cuda()

In [8]:
y_train

tensor([136.,  90., 112.,  ...,  18.,  41., 114.])

In [9]:
#y_train = y_train.unsqueeze(-1)
y_train = y_train.type(torch.LongTensor).cuda()

In [10]:
layer1 = torch.nn.Linear(256,1042,bias=True)
layer2 = torch.nn.Linear(1042,512,bias=True)
layer3 = torch.nn.Linear(512,198,bias=True)
relu = torch.nn.ReLU()
model = torch.nn.Sequential(layer1,relu,layer2,relu,layer3).cuda()

In [11]:
loss = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [12]:
for stop in range(3001):
    
    # 그래디언트 초기화
    optimizer.zero_grad()
    # Forward 계산
    hypothesis = model(x_train)
    # Error 계산
    cost = loss(hypothesis, y_train)
    # Backward 계산 
    cost.backward()
    # 가중치 갱신
    optimizer.step()

    if stop % 500 == 0:
        print(stop, cost.item())

0 5.283855438232422
500 0.000899750622920692
1000 0.0002364932297496125
1500 0.00010560151713434607
2000 5.778974809800275e-05
2500 3.518905214150436e-05
3000 2.285565460624639e-05


In [13]:
predict = model(x_test)

In [14]:
predict = torch.argmax(predict,dim=1)

In [15]:
predict[10:50]

tensor([ 91, 177, 110, 114, 168,  76,  17,  76,  38, 112, 126,  83,  57,  76,
          8,  23,  21,  17,  76, 126,  17, 114, 159, 178, 166, 114, 114, 126,
         62,  68,  65,  10, 126, 179,  91,  83,  38,  40,  33, 167],
       device='cuda:0')

In [16]:
submit['label'] = predict.cpu()
submission=submit.astype(np.str_)
submission.to_csv('submission.csv',mode='w',header=True,index=False)